Foundation API Information
--

In [1]:
# initialize libraries
import requests
from requests import api
from requests.models import Response
import json
import bcrypt
from datetime import datetime, timezone
from pprint import pprint
from tabulate import tabulate
import csv
import os
from dotenv import load_dotenv


In [2]:
# Define the base URL and API key for the Foundation API
load_dotenv()

base_url = os.getenv('BASE_URL')
api_key = os.getenv('API_KEY')
api_key_value = os.getenv('API_KEY_VALUE')


In [3]:
print(f"Base URL: {base_url}")
print(f"API Key: {api_key}")
print(f"API Key Value: {api_key_value}")

Base URL: https://knobbestaging.foundation.litera.com/
API Key: knobbe-apikey
API Key Value: 0f500c90-9df1-4bd4-b789-b89f1a1bcf65


In [4]:
def get_foundation_headers():
    # Use bcrypt to hash the API key with a timestamp per Litera documentation
    instant_timestamp = datetime.now(timezone.utc).strftime("%Y-%m-%dT%H:%M:%S.%fZ")[:-1] + "0Z"  
    #instant_timestamp = datetime.utcnow().strftime("%Y-%m-%dT%H:%M:%S.%fZ")[:-1] + "0Z"  # Ensures 7 places after decimal
    combined = api_key_value + "|" + instant_timestamp
    salt = bcrypt.gensalt()
    hashed = bcrypt.hashpw(combined.encode('utf-8'), salt)
    hashed_key = hashed, instant_timestamp
    result = {'x-foundation-api-key': api_key, 'x-foundation-timestamp': hashed_key[1], 'x-foundation-api-auth': hashed_key[0], "accept":"application/json"}
    return result

In [9]:
# Function to get metadata from the Foundation metadata API endpoint
def get_foundation_metadata():
    headers = get_foundation_headers()
    url = f"{base_url}api/v1/application/metadata"
    response = requests.get(url, headers=headers, verify=False)
    
    if response.status_code == 200:
        metadata = response.json()
        # pprint(metadata)
        return metadata
    else:
        print(f"Error: {response.status_code} - {response.text}")
        return None

In [ ]:
# load metadata
metadata = get_foundation_metadata()
print(f"Metadata: {metadata}")
